# Scrap like a Ninja

In order to be a real ninja scraper, you will have to build a custom selenium driver 😎

Writing a custom Selenium driver offers several benefits, particularly when dealing with dynamic and complex web pages that may have measures to detect and block automated scraping. It provides a level of control and customization that is necessary for effective scraping of modern web applications. 

## Our goal 🚀

*__Marche à suivre__*

L'URL à utiliser pour cet exercice est :
https://www.welcometothejungle.com/fr/jobs?page=1&refinementList%5Bprofession_name.fr.Tech%5D%5B%5D=Data%20Analysis&refinementList%5Bcontract_type_names.fr%5D%5B%5D=CDI

On peut remarquer que :

1. Il y a plusieurs pages de résultats, que l'on peut parcourir simplement **en changeant `page=k` dans l'URL**. Il y a **30 postes proposés par page** de résultats.
<br><br>
2. Welcome to the jungle a implémenté des mesures anti-scraping. En particulier, une partie du HTML est cachée lorsque l'on requête la page avec  `requests`. Il est indispensable de **commencer à scroller** la page pour lancer le code JavaScript qui révèle le contenu caché.
<br><br>
$\rightarrow$ Pour résoudre ce problème, on ne peut se contenter de BeautifulSoup. Il faut **Simuler le comportement d'une vraie personne** qui parcourt la page avec sa souris, c'est donc **Selenium** qu'il nous faut
<br><br>
$\rightarrow$ Voilà une fonction qui permet de simuler un scroll de page jusqu'à la ième offre d'emploi :
<br><br>

```python
def scroll(driver, i):
        scroll_delta = int(250)
        scroll_delta += 140*i
        driver.execute_script("window.scrollBy(0, "+ str(scroll_delta) + ")")
```
<br>

3. Une autre mesure anti-scraping concerne les noms de classes, les ids et même les liens vers des images dans le code HTML. Tous ces noms sont aléatoires (ex:`class="sc-1flb27e-5 cdtiMs"`) et changent à chaque chargement de la page.<br><br>
$\rightarrow$ Une bonne nouvelle quand même : toutes les classes ne sont pas aléatoires, certaines restent fixes. Pour les noms aléatoires, certaines lettres du nom sont fixes également. On peut donc toujours utiliser des similarités pour désigner certains tags spécifiques (ex : le tag header, contenant le nombre total de résultats, commence toujours par "hd").<br><br>
$\rightarrow$ Pour exploiter cette faille, il est conseillé d'utiliser la méthode Selenium **`find_elements_by_css_selector()`** pour désigner des tags précis, car cette méthode permet de d'identifier un tag par un texte partiel (ex: `driver.find_elements_by_css_selector("header[class^='hd']")` pour toutes les classes de headers qui commencent par "hd").
<br><br>
4. Au bout du compte, on souhaite sauvegarder le contenu de chaque offre d'emploi dans un fichier .txt.<br><br>
$\rightarrow$ Il va donc falloir cliquer sur chaque offre d'emploi avec la méthode **`.click()`** de Selenium. Pour chaque offre d'emploi, le contenu de l'offre est stocké dans un dictionnaire à l'intérieur d'un tag `<script>`. On peut utiliser la méthode  **`json.loads()`** pour manipuler ce dictionnaire. On peut finalement l'enregistrer en .txt avec les fonctions **`open()`** et **`.write`**.
5. Sauvegarder le contenu de chaque offre d'emploi dans une database postgres puis mongodb. <br><br>
$\rightarrow$Utiliser un `dataframe` comme structure intermédiaire. <br>
$\rightarrow$Quel est le problème de postgres? <br>
$\rightarrow$Quel est la différence avec mongodb?

## Understand proxies 

A proxy server is a server application that acts as an intermediary between a client requesting a resource and the server providing that resource, more infos on [wikipedia](https://en.wikipedia.org/wiki/Proxy_server)

![](https://upload.wikimedia.org/wikipedia/commons/thumb/9/91/Proxy_concept_fr.svg/752px-Proxy_concept_fr.svg.png)


In [1]:
#install free proxy tool from : https://github.com/jundymek/free-proxy
#!pip install free-proxy

You can also find free proxies here : https://free-proxy-list.net 

Or more pro solutions for goods tool like Captia bypass here : https://www.zenrows.com/solutions/bypass-captcha

In [2]:
from fp.fp import FreeProxy
import requests
from bs4 import BeautifulSoup

In [3]:
proxy = FreeProxy(country_id=['FR']).get(); proxy

'http://162.248.225.122:80'

In [4]:
proxies = {'http': proxy} 
response = requests.get('http://httpbin.org/ip', proxies=proxies) 
print(response.json()['origin']) # our proxy !!


162.248.225.122


In [5]:
proxy_list = [FreeProxy(country_id=['FR']).get() for x in range(3)]; set(proxy_list)

{'http://162.248.225.131:80', 'http://162.248.225.224:80'}

Great we have now a different IP address, at least the server detect an other ip and not our public router IP 🧙🏼‍♂️

### Headers 

Now let's get deeper a little with our request header in order to fool our target with `User-Agent` (abbreviated as UA). A user agent is a string that a web browser sends to a web server identifying itself. This string contains details about the browser type, rendering engine, operating system, and sometimes device type. In web scraping, the user agent plays a crucial role for several reasons:

- Browser Identification: The user agent tells the server what kind of browser is making the request. Different browsers may support different features or render web pages differently.
- Device and OS Identification: The user agent can also indicate the operating system and the device (desktop, mobile, tablet, etc.), which can affect how web content is delivered.

Let's see a basic example of the informations the target site will get if we use Python Requests or cURL without any modifications.

In [6]:
response = requests.get('http://httpbin.org/headers') 
print(response.json()['headers']['User-Agent']) 
# python-requests/2.25.1


python-requests/2.31.0


In [7]:
!curl http://httpbin.org/headers # { ... "User-Agent": "curl/7.74.0" ... }

{
  "headers": {
    "Accept": "*/*", 
    "Host": "httpbin.org", 
    "User-Agent": "curl/7.64.1", 
    "X-Amzn-Trace-Id": "Root=1-65786377-28e3d9436665d7ea0ab8992e"
  }
}


In [8]:
#try a custom user-agent
headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36"} 
response = requests.get('http://httpbin.org/headers', headers=headers) 
print(response.json()['headers']['User-Agent']) # Mozilla/5.0 ...


Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36


In [9]:
#more user-agent, thanks chatgpt 🤓
import random
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148',
    'Mozilla/5.0 (Linux; Android 11; SM-G960U) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.72 Mobile Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0',
    'Mozilla/5.0 (iPad; CPU OS 13_5 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Mobile/15E148 Safari/604.1',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/604.1.34 (KHTML, like Gecko) Edge/90.0.818.56',
    'Mozilla/5.0 (Linux; Android 10; SM-A505FN) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Mobile Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15',
    'Mozilla/5.0 (Linux; Android 11; Pixel 3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.101 Mobile Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1',
    'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:88.0) Gecko/20100101 Firefox/88.0'
]

user_agent = random.choice(user_agents) 
headers = {'User-Agent': user_agent} 
response = requests.get('https://httpbin.org/headers', headers=headers) 
print(response.json()['headers']['User-Agent']) 
# Mozilla/5.0 (iPhone; CPU iPhone OS 12_2 like Mac OS X) ...

Mozilla/5.0 (Linux; Android 10; SM-A505FN) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Mobile Safari/537.36


If you take a closer look to our request [here](http://httpbin.org/headers) you will see the entier header look like this :

```bash
{
  "headers": {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", 
    "Accept-Encoding": "gzip, deflate", 
    "Accept-Language": "fr-fr", 
    "Host": "httpbin.org", 
    "Upgrade-Insecure-Requests": "1", 
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.6.1 Safari/605.1.15", 
    "X-Amzn-Trace-Id": "Root=1-6572fed4-5a7b863b4842def83f9030c4"
  }
}
```

In [10]:
headers_list = [
    {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.9",
        "Host": "httpbin.org",
        "Sec-Ch-Ua": "\"Chromium\";v=\"92\", \" Not A;Brand\";v=\"99\", \"Google Chrome\";v=\"92\"",
        "Sec-Ch-Ua-Mobile": "?0",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    },
    {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.5",
        "Host": "httpbin.org",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0"
    },
    {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.5",
        "Host": "httpbin.org",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1"
    },
    {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-GB,en;q=0.5",
        "Host": "httpbin.org",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:54.0) Gecko/20100101 Firefox/54.0"
    },
    {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.9",
        "Host": "httpbin.org",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.2 Safari/605.1.15"
    }
]

headers = random.choice(headers_list) 
response = requests.get('https://httpbin.org/headers', headers=headers) 
print(response.json()['headers']['User-Agent']) 

Mozilla/5.0 (Windows NT 6.1; WOW64; rv:54.0) Gecko/20100101 Firefox/54.0


## Custom Selenium driver 

In [11]:
import os
# Set the environment variable for SSL certificate
certi_path = "/Users/mac/.pyenv/versions/3.7.0/lib/python3.7/site-packages/certifi/cacert.pem"
os.environ['REQUESTS_CA_BUNDLE'] = certi_path
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options


def initialize_driver(headers_list, proxy_list):
    options = Options()
    #select a random user-agent from the list
    user_agent = random.choice(headers_list)["User-Agent"]
    options.add_argument(f"user-agent={user_agent}")
    
    #select a random proxy from the list
    proxy = random.choice(proxy_list)
    if proxy:
        options.add_argument(f"--proxy-server={proxy}")
    
    #add some common options
    options.add_argument("--headless")
    options.add_argument("--disable-extensions")
    options.add_argument("--ignore-certificate-errors")

    #initialize Chrome WebDriver with the specified options
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    #set implicit wait of 10sec
    driver.implicitly_wait(10)

    return driver

# Example usage
custom_driver = initialize_driver(headers_list, proxy_list)

In [33]:
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


import os
# Set the environment variable for SSL certificate
certi_path = "/Users/mac/.pyenv/versions/3.7.0/lib/python3.7/site-packages/certifi/cacert.pem"
os.environ['REQUESTS_CA_BUNDLE'] = certi_path

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

# Initialize Chrome options
options = Options()
#options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
#disable the browser
#options.add_argument("--headless")
options.add_argument("--disable-extensions")
options.add_argument("--ignore-certificate-errors")
#options.add_argument(f'--proxy-server={proxy}')

# Initialize Chrome WebDriver with the specified options
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)




## Get the main page 

Our goal here is very simple: writing a python function called `MainPage` to get this url : `https://www.welcometothejungle.com/fr/jobs?page=1&configure%5Bfilters%5D=website.reference%3Awttj_fr&configure%5BhitsPerPage%5D=30&aroundQuery=France&refinementList%5Boffice.country_code%5D%5B%5D=FR&refinementList%5Bcontract_type_names.fr%5D%5B%5D=CDI&refinementList%5Bcontract_type_names.fr%5D%5B%5D=Stage&query=%22data%20analyst%22&range%5Bexperience_level_minimum%5D%5Bmin%5D=0&range%5Bexperience_level_minimum%5D%5Bmax%5D=1`

and do a sleep of 3 seconds. 

```python 
def MainPage(driver, url):
    '''Go the the first page and sleep(3)'''
    #code here 
```

In [34]:
def MainPage(driver, url):
    '''Go the the first page'''
    driver.get(url)
    time.sleep(3)

## Get the number of offers per page


Write a python function who return the number of job offer in a page 

```python
def nbOffers(driver):
    try:
        #code here 

    except Exception as e:
        print("An error occurred in NB_OFFER:", str(e))
        return 0  # Or handle the exception as needed


```

Example usage of the `nbOffers` function : 

```python 
url = f"https://www.welcometothejungle.com/fr/jobs?page=1&configure%5Bfilters%5D=website.reference%3Awttj_fr&configure%5BhitsPerPage%5D=30&aroundQuery=France&refinementList%5Boffice.country_code%5D%5B%5D=FR&refinementList%5Bcontract_type_names.fr%5D%5B%5D=CDI&refinementList%5Bcontract_type_names.fr%5D%5B%5D=Stage&query=%22data%20analyst%22&range%5Bexperience_level_minimum%5D%5Bmin%5D=0&range%5Bexperience_level_minimum%5D%5Bmax%5D=1"
MainPage(driver, url)
nb_offers = nbOffers(driver)
```

Then write a python function to get all the jobs post :

```python 
def nbOffers_tot(driver):
    try:
        #code here

    except Exception as e:
        print("An error occurred in NB_OFFER TOTAL:", str(e))
        return 0  # Or handle the exception as needed
```

In [35]:
def nbOffers(driver):
    try:
        # Wait for the job offer elements to be present on the page
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li[class$='ais-Hits-list-item']"))
        )
        
        # Find all elements with the specified class
        job_offers = driver.find_elements(By.CSS_SELECTOR, "li[class$='ais-Hits-list-item']")
        #print(job_offers)
        # The total number of job offers is the number of elements found
        return len(job_offers)

    except Exception as e:
        print("An error occurred in NB_OFFER:", str(e))
        return 0  # Or handle the exception as needed




In [36]:

def nbOffers_tot(driver):
    try:
        # Wait for the div with the specified attribute to be available
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//div[@data-testid='jobs-search-results-count']"))
        )
        #return the text of the element
        return int(element.text)

    except Exception as e:
        print("An error occurred in NB_OFFER TOTAL:", str(e))
        return 0  # Or handle the exception as needed




In [37]:
#ouverture de la page et récupération du nombre d'offres
url = f"https://www.welcometothejungle.com/fr/jobs?page=1&configure%5Bfilters%5D=website.reference%3Awttj_fr&configure%5BhitsPerPage%5D=30&aroundQuery=France&refinementList%5Boffice.country_code%5D%5B%5D=FR&refinementList%5Bcontract_type_names.fr%5D%5B%5D=CDI&refinementList%5Bcontract_type_names.fr%5D%5B%5D=Stage&query=%22data%20analyst%22&range%5Bexperience_level_minimum%5D%5Bmin%5D=0&range%5Bexperience_level_minimum%5D%5Bmax%5D=1"
MainPage(driver, url)
nb_offers = nbOffers(driver)
nb_offerst = nbOffers_tot(driver)
print(f"\nNumbers offers tot : {nb_offerst} \nNumber of offers per page : {nb_offers}")


Numbers offers tot : 66 
Number of offers per page : 30


## `Click` and `getText` functions

Write a python function who click on a given selenium element : 

```python 

def Click(driver, pos):
    '''Click on the link'''
    try:
        #code here
    except Exception as e:
        print("An error occurred in CLICK:", str(e))
        return 0  # Or handle the exception as needed
```

Then write a function who get the text with beatifulsoup of a job post, save it into a list and a txt file :
```python
def GetText(driver, jobs):
    sleep(3)
    #code here
    try:
        
    except Exception as e:
        print(f"Error HTML PARSING: {e}")


```

In [40]:


def scroll(driver, i):
    scroll_delta = int(250)
    scroll_delta += 140*i
    driver.execute_script("window.scrollBy(0, "+ str(scroll_delta) + ")")

    


def Click(driver, pos):
    '''Click on the link'''
    try:
        elements = driver.find_elements(By.CSS_SELECTOR, "li[class$='ais-Hits-list-item']")
        target_element = elements[pos]
        #click with javascript
        driver.execute_script("arguments[0].scrollIntoView();", target_element)
        target_element.click()
    except Exception as e:
        print("An error occurred in CLICK:", str(e))
        return 0  # Or handle the exception as needed


def GetText(driver, jobs):
    time.sleep(3)
    #print(str(driver.current_url))
    data = requests.get(driver.current_url)
    soup = BeautifulSoup(data.text, 'html.parser')
    try:
        info = soup.find("main")
        if info is not None:
            info = info.find_all('script')
            job = json.loads(str(info[0].string))
            faq = json.loads(str(info[1].string))
            job['FAQPage'] = faq['mainEntity']
        else:
            info = soup.find("div").find_all('script')
            job = json.loads(str(info[0].string))
            faq = json.loads(str(info[1].string))
            job['FAQPage'] = faq['mainEntity']

        jobs.append(job)
        f = open('wttj.txt', 'a')
        f.write(str(job))
        print(f'write job : {driver.current_url}')
        f.write('\n')
        f.close()
    except Exception as e:
        print(f"Error HTML PARSING: {e}")


## Put it into a loop 👨‍🍳👩‍🍳

Write a simple loop over page in order to put all the jobs into a list named `jobs` you can add a `break` statement for the debuging part, it can be long 🤓

In [41]:
# boucle de scraping
counter = 1
jobs = []
for i in range (1, (nb_offerst // 30) + 1):
    url = f"https://www.welcometothejungle.com/fr/jobs?page={i}&configure%5Bfilters%5D=website.reference%3Awttj_fr&configure%5BhitsPerPage%5D=30&aroundQuery=France&refinementList%5Boffice.country_code%5D%5B%5D=FR&refinementList%5Bcontract_type_names.fr%5D%5B%5D=CDI&refinementList%5Bcontract_type_names.fr%5D%5B%5D=Stage&query=%22data%20analyst%22&range%5Bexperience_level_minimum%5D%5Bmin%5D=0&range%5Bexperience_level_minimum%5D%5Bmax%5D=1"
    for j in range(0,30):
        MainPage(driver, url)
        scroll(driver, j)
        Click(driver, j)
        GetText(driver, jobs)
        counter += 1
        print(f"\n{counter}/{int(nb_offerst)}", end='')
        break

write job : https://www.welcometothejungle.com/fr/companies/matera/jobs/data-analyst-stage-de-6-mois_paris?q=8a940671e03319920400c3a073bdcf37&o=2228430

2/66write job : https://www.welcometothejungle.com/fr/companies/pwc/jobs/data-analyst-deals-m-a-cdi-h-f_paris_PF_GPPgd6A?q=b9ec9f46e728f4465929db3fb001b7ec&o=2262089

3/66

## Clean it and insert into PostgresSQL database

Here our mission is simple clean the data as least a little in order to insert them into a PostgresSQL database. 

0. Transform our job list into a pandas Dataframe
1. Clean the text inside the `description` column
- Write a function called `extract_salary_info()` who split the `baseSalary` columns into `['minSalary', 'maxSalary', 'currency', 'salaryUnit']` 
- Extract the `name` variable inside the `hiringOrganization` column
- Extract the `addressLocality` variable inside the `JobLocation` column
- Drop the columns `['@context','baseSalary','educationRequirements','experienceRequirements','FAQPage']`


In [ ]:
jobs_df = pd.DataFrame(jobs); jobs_df.head()

In [ ]:
# Function to extract text from HTML
def extract_text_from_html(html_content):
    if pd.isna(html_content):
        return None
    soup = BeautifulSoup(html_content, 'html.parser')
    return soup.get_text(separator=' ', strip=True)

# Apply function to description column
jobs_df['description'] = jobs_df['description'].apply(extract_text_from_html)


In [ ]:
def extract_salary_info(salary_data):
    if pd.isna(salary_data):
        return [None, None, None, None]
    currency = salary_data.get('currency', None)
    value = salary_data.get('value', {})
    min_value = value.get('minValue', None)
    max_value = value.get('maxValue', None)
    unit_text = value.get('unitText', None)
    return [min_value, max_value, currency, unit_text]

# Apply the function to baseSalary column and create new columns
jobs_df[['minSalary', 'maxSalary', 'currency', 'salaryUnit']] = jobs_df['baseSalary'].apply(lambda x: pd.Series(extract_salary_info(x)))


In [ ]:
jobs_df

## Databases insertion

Our goal in this part is to insert our result data into a postgres database 😎

We will use docker to run our postgres database in a simple way  with this command :

```bash
docker run --name posttest -d -p 5432:5432 -e POSTGRES_PASSWORD=fred postgres:alpine
```

You mission is simple : write data to the database in a `job_table` table !

You can use this sample code to connect your database :
```python 
from sqlalchemy import create_engine

# Database credentials
user = 'postgres'
password = 'fred'
host = '0.0.0.0'  # or the IP if your PostgreSQL server is running elsewhere
port = '5432'       # default port for PostgreSQL used by our docker above
db = 'postgres'

# Create the connection
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db}')
```

Then write a little script who connect to the database and list all the tables inside then perform a verification query (e.g., selecting the first 5 rows)

In [ ]:
!pip install psycopg2 sqlalchemy

In [ ]:
!docker run --name posttest -d -p 5432:5432 -e POSTGRES_PASSWORD=fred postgres:alpine

In [ ]:
from sqlalchemy import create_engine

# Database credentials
user = 'postgres'
password = 'fred'
host = '0.0.0.0'  # or the IP if your PostgreSQL server is running elsewhere
port = '5432'       # default port for PostgreSQL
db = 'postgres'

# Create the connection
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db}')



In [ ]:
engine

In [ ]:
jobs_df['hiringOrganization'] = jobs_df['hiringOrganization'].apply(lambda x: x['name'])

In [ ]:
jobs_df['jobLocation'] = jobs_df['jobLocation'].apply(lambda x: x[0]['address']['addressLocality'])

In [ ]:
jobs_df = jobs_df.drop(['@context','baseSalary','educationRequirements','experienceRequirements','FAQPage'], axis=1)

In [ ]:
jobs_df.head()

In [ ]:
#write data to the database in `job_table` table
jobs_df.to_sql('job_table', engine, if_exists='replace', index=False)  # Choose 'append' if you don't want to overwrite

In [ ]:
import psycopg2

#connect to the PostgreSQL database
conn = psycopg2.connect(dbname='postgres', user=user, password=password, host=host, port=port)

#create a cursor object
cursor = conn.cursor()

#list all tables in the database
cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'")
tables = cursor.fetchall()
print("Tables in the database:")
for table in tables:
    print(table)

#check if job_table is created
if ('job_table',) in tables:
    print("\nTable 'job_table' exists.")
else:
    print("\nTable 'job_table' does not exist.")

#perform a verification query (e.g., selecting the first 5 rows)
query = "SELECT * FROM job_table LIMIT 5"
df = pd.read_sql(query, conn)
print("\nFirst 5 rows of 'job_table':")
print(df)


### MongoDB

Same mission with mongo :

```
docker run -d --name example-mongo -p 27017:27017 mongo
```

Connect the mongo database and do a dummy query like *find the number of documents where the currency is 'EUR'*

In [ ]:
#!pip install pymongo

In [ ]:
!docker run -d --name example-mongo -p 27017:27017 mongo

In [ ]:
from pymongo import MongoClient

#connect to the MongoDB server (default is localhost on port 27017)
client = MongoClient('0.0.0.0', 27017)

#access the database (create it if it doesn't exist)
db = client['job_database']

#access the collection (similar to a table in relational databases)
collection = db['job_collection']


In [ ]:
#convert DataFrame to dictionary format and insert into our MongoDB database
collection.insert_many(jobs_df.to_dict('records'))

In [ ]:
#find the number of documents where the currency is 'EUR'
count = 0
for job in collection.find({"currency": "EUR"}):
    count+=1
print(count)